# Rater Reliability Notebook
This notebook will require edits to work, but has been left in the repo in case you'd like to adjust it.

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import operator

In [ ]:
#Fill in Participant Number Here
participant_number = '1079' 
df = pd.read_csv("summer_2019/1079/output/responses.csv")

In [ ]:
df['count'] = df.groupby(['file_name']).file_name.transform('count')

In [ ]:
df_an1 = df[df['count'] == 1]
val_counts = df['count'].value_counts()
for k, v in val_counts.items():
    val_counts[k] = int(v)/int(k)
    print(int(k), ": ", val_counts[k], "clips")

In [ ]:
df_an1['Language'].value_counts()

In [ ]:
unsure = df_an1[df_an1['Language'] == 'Unsure']

In [ ]:
languages = set(['Spanish', 'Quechua', 'Mixed'])
lang_ratio = df_an1[df_an1['Language'].isin(languages)]

### Clips Annotated Twice

In [ ]:
df_an2 = df[df['count'] == 2]

In [ ]:
grouped = df_an2.groupby('file_name')

In [ ]:
intrarater_dict = defaultdict(list)
interrater_dict = defaultdict(list)
for n, group in grouped:
    annotators = list(group['annotator'])
    group = pd.DataFrame(group).reset_index()
    langs = list(group['Language'])
    if annotators[0] == annotators[1]:  #SAME
        annotator = annotators[0]
        if langs[0] == langs[1]:
            intrarater_dict[annotator].append(1)
            if langs[0] == 'Unsure':
                unsure = unsure.append(group[0:1], ignore_index=False, verify_integrity=False, sort=None)
            else:
                lang_ratio = lang_ratio.append(group[0:1])
        else:
            if any(elem in languages for elem in langs):
                intrarater_dict[annotator].append(0)
                if 'Unsure' in langs:
                    unsure = unsure.append(group[0:2])
    else: #Different
        if annotators[0] != annotators[1] and langs[0] == langs[1]:
            interrater_dict[annotators[0]].append(1)
            interrater_dict[annotators[1]].append(1)
            lang_ratio = lang_ratio.append(group[0:1])
        else:
            if annotators[0] != annotators[1] and any(elem in languages for elem in langs):
                interrater_dict[annotators[0]].append(0)
                interrater_dict[annotators[1]].append(0)
                if any(elem == 'Unsure' for elem in langs):
                    unsure = unsure.append(group[0:2])


## Clips Annotated Thrice+

In [ ]:
valCount = {'a':2, 'b':1}
annToLangs = [('k', 'a'), ('l', 'b'), ('l', 'a')]
anns = ['k', 'l', 'l']
majorityLang, majorityCount = max(valCount.items(), key=operator.itemgetter(1))
allMajorityLangs = [key for key in valCount.keys() if valCount[key]==majorityCount]
d1 = defaultdict(list)
for ann, lang in annToLangs:
    if lang == majorityLang:
           d1[ann].append(1)
    else:
           d1[ann].append(0)

In [ ]:
df_an3 = df[df['count'] >= 3]
grouped3 = df_an3.groupby('file_name')
for n, group in grouped3:
    annotators = list(group['annotator'])
    langs = list(group['Language'])
    annToLangs = zip(annotators, langs)
    uniqueAnns = set(annotators)
    valCount = group['Language'].value_counts()
    majorityLang, majorityCount = max(valCount.items(), key=operator.itemgetter(1))
    allMajorityLangs = [key for key in valCount.keys() if valCount[key]==majorityCount]
    if len(allMajorityLangs) == 1 and majorityLang in languages:
        lang_ratio = lang_ratio.append(group[0:1])
        if len(uniqueAnns) == 1:
            intrarater_dict[uniqueAnns[0]].append(1)
        else:
            for ann, lang in annToLangs:
                if lang == majorityLang:
                    interrater_dict[ann].append(1)
                else:
                    interrater_dict[ann].append(0)


## Interrater Ratio

In [ ]:
interrater_df = pd.DataFrame(columns=['Annotator', 'InterRatio'])
for ann in interrater_dict:
    inter_ratio = sum(interrater_dict[ann])/len(interrater_dict[ann])
    print(ann, "Ratio: " , inter_ratio)
    print("Number of clips: ", len(interrater_dict[ann]))
    interrater_df = interrater_df.append({'Annotator': ann, 'IntraRatio':inter_ratio}, ignore_index=True)

## Intrarater Ratio

In [ ]:
intrarater_df = pd.DataFrame(columns=['Annotator', 'IntraRatio'])
for ann in intrarater_dict:
    intra_ratio = sum(intrarater_dict[ann])/len(intrarater_dict[ann])
    print(ann, "Ratio: " , intra_ratio)
    print("Number of clips: ", len(intrarater_dict[ann]))
    intrarater_df = intrarater_df.append({'Annotator': ann, 'IntraRatio':intra_ratio}, ignore_index=True)

## Convert to CSV

In [ ]:
interrater_df.to_csv(participant_number + '_interrater.csv')
intrarater_df.to_csv(participant_number  + '_intrarater.csv')
lang_ratio.to_csv(participant_number + '_bilingualratio.csv')
unsure.to_csv(participant_number  + '_relisten.csv')